In [1]:
### imports
import json
import import_ipynb
import ipywidgets as widgets
import tensorflow as tf
from sklearn.model_selection import train_test_split
from ipywidgets import interact, interactive, fixed, interact_manual
from objects import *
from video import *

importing Jupyter notebook from objects.ipynb
importing Jupyter notebook from video.ipynb
importing Jupyter notebook from landmarks.ipynb
importing Jupyter notebook from plotting.ipynb


## Metadata

In [2]:
Meta = metadata('supplemental')
Encode, Decode = Meta.encode, Meta.decode

In [3]:
%pprint

Pretty printing has been turned OFF


In [4]:
Decode

{0: 'a', 1: 'b', 2: 'c', 3: 'cc', 4: 'd', 5: 'dd', 6: 'e', 7: 'ee', 8: 'f', 9: 'ff', 10: 'g', 11: 'gg', 12: 'h', 13: 'i', 14: 'j', 15: 'k', 16: 'l', 17: 'll', 18: 'm', 19: 'mm', 20: 'n', 21: 'nn', 22: 'o', 23: 'oo', 24: 'p', 25: 'pp', 26: 'q', 27: 'r', 28: 'rr', 29: 's', 30: 'ss', 31: 't', 32: 'tt', 33: 'u', 34: 'v', 35: 'w', 36: 'x', 37: 'y', 38: 'z', 39: 'zz'}

## Selecting datafiles

In [38]:
file = str(Meta.files[0])
Datafile = datafile(Meta, file)

## Selecting videos

In [6]:
vid = Datafile.videos[1]
V = video(Datafile, vid)

In [7]:
print(V)

 Sequence: 731357147
 Phrase: mom made her a turtleneck
 Signer: 111
 Frames: 124
 Dominant Hand: right
 Percentage: 39.52


In [8]:
#Good = []
#for i,vid in enumerate(Datafile.videos):
    #V = video(Datafile, vid)
    #p = percentage(V)
    #if p == 100.00: 
        #Good += [vid]
    #print(i,p,len(Good))

In [9]:
Good = np.array([731388395, 733037660, 733083339, 733880365, 735770695, 737406450,
       740539260, 742144914, 742612327, 742769082, 742979308, 743509352,
       744720263, 745277714, 745817010, 746072464, 748370228, 748683418,
       748895323, 750445797, 751067574, 751959745, 752039037, 752973672,
       755835802, 756981660, 757335344, 757918060, 758005970, 758610320,
       758757555, 759275436, 759824943, 762418872, 762613712, 762732088,
       763196596, 764432761, 765939582, 766090398, 767151702, 768694051,
       768877329, 769996737, 770150821])

In [10]:
V = video(Datafile, Good[0])

In [11]:
f = show_hands(V)
interact(f, frame=widgets.IntSlider(min=0, max=len(V.frames)-1, step=1, value=0, layout=widgets.Layout(width='1000px')))

interactive(children=(IntSlider(value=0, description='frame', layout=Layout(width='1000px'), max=212), Output(…

<function show_hands.<locals>.show_frame at 0x2b0dff130>

In [39]:
# load Data
with open('LabeledData/LabeledData' + str(Datafile.id) + '.json', 'r') as f:
    train = json.load(f)

X, y = [], []
# loop over file ids
for file in train:
    # create datafile
    Df = datafile(Meta, file)
    videos = train[file]
    for vid in videos:
        
        # load date
        V = video(Df, int(vid))
        N, p = normal_hand_data(V)
        pN = videos[vid]
        
        # loop
        for i in range(len(N)):
            if pN[i] != '?':
                X += [ N[i] ]
                y += [ Encode[ pN[i] ] ]

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

In [31]:
# Model
k = 40
M = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, 3)),
    tf.keras.layers.Dense(6*k, activation='relu'),
    tf.keras.layers.Dense(k)])

# Compile Model
M.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [34]:
# fit model
# M.fit( X, y, epochs = 400, verbose=0 )
M.fit( X_train, y_train, epochs = 400, verbose=0 )

In [40]:
Loss, Acctrain = M.evaluate(X, y, verbose=2)
print('accuracy:', Acctrain)

177/177 - 0s - loss: 1.4730 - accuracy: 0.8012 - 132ms/epoch - 747us/step
accuracy: 0.8011685609817505


In [35]:
Loss, Acctrain = M.evaluate(X_train, y_train, verbose=2)
Loss, Acctest = M.evaluate(X_test, y_test, verbose=2)
print('\nTrain accuracy:', Acctrain)
print('\nTest accuracy:', Acctest)

138/138 - 0s - loss: 0.2051 - accuracy: 0.9451 - 57ms/epoch - 413us/step
59/59 - 0s - loss: 0.7341 - accuracy: 0.8570 - 32ms/epoch - 547us/step

Train accuracy: 0.9450649619102478

Test accuracy: 0.8569909334182739


In [ ]:
Label(V,M)

In [19]:
#with open("LabeledData/LabledData40.json", 'r') as f:
    #pickles = json.load(f)

In [20]:
# pickle function
def pickle(Dict, V):
    seq_id = str(V.id)
    Frames = V.frames
    Dict[seq_id] = [ label(V,i) for i in Frames ]

In [21]:
# pickling good movies
pepper = {}
for vid in Good:
    V = video(Datafile, vid)
    Label(V,M)
    pickle( pepper, V )

In [25]:
D = { str(Datafile.id) : pepper }

In [27]:
Datafile.id

'86446671'

In [28]:
with open('LabeledData_' + str(Datafile.id) + '.json', 'w') as f:
    json.dump(D, f, indent=2)